### Exploring data from datahub.io

Acquiring data from [here](https://github.com/datasets/covid-19) and maybe soon be able to automate the integration of that data with wikidata.

Just some things to think about (jvfe):
- How to properly reference the data? Choose [datahub.io](https://datahub.io/core/covid-19) as the reference?
    - They aggregate it from various sources
    
    
- I've acquired the country outbreak items via the following query and modified it slightly to better merge the items.
```
SELECT ?item ?itemLabel ?countryid ?countryidLabel
WHERE 
{
  ?item p:P31 ?statement. 
      ?statement ps:P31 wd:Q3241045. 
      ?statement pq:P642 wd:Q84263196.
      ?statement pq:P3005 ?countryid.
      ?countryid wdt:P31 wd:Q6256.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
```

In [1]:
%load_ext pycodestyle_magic

In [2]:
%flake8_on

In [3]:
import pandas as pd

In [4]:
from datetime import date, time, timedelta
yesterday = date.today() - timedelta(days=1)
today = date.today()

yesterday_table = yesterday.strftime("%Y-%m-%d")
today_table = today.strftime("%Y-%m-%d")


In [5]:
countries = pd.read_csv("https://raw.githubusercontent.com/datasets/covid-19/master/data/countries-aggregated.csv")
wdt_items = pd.read_csv("../data/country_outbreaks.csv")

1:80: E501 line too long (115 > 79 characters)


In [6]:
full = pd.merge(countries, wdt_items, on="Country")
full.head(3)

,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid
0,2020-01-22,Afghanistan,0,0,0,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889
1,2020-01-23,Afghanistan,0,0,0,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889
2,2020-01-24,Afghanistan,0,0,0,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889


In [7]:
from datetime import datetime
# Most recent data seems to be from the day before

query = "Date == @yesterday_table"
recent = full.query(query)

# that does not happen aways, though.


dates_in_full = [datetime.strptime(date, "%Y-%m-%d") for date in full["Date"]]
most_recent_date = max(dates_in_full).strftime("%Y-%m-%d")

# pd.query did not work
recent = full[full["Date"] == most_recent_date]

recent.head(2)

,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid
144,2020-06-14,Afghanistan,24766,4725,471,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889
289,2020-06-14,Algeria,10919,7606,767,Q87202921,2020 coronavirus pandemic in Algeria,Q262


In [8]:
# The following countries appear to be updated
# manually from more specific sources.
idx = recent['Country'].isin(['US', 'United Kingdom', 'France', 'Sweden',
                              'Brazil', 'Netherlands', 'China', 'Italy',
                              'Spain', 'Germany', 'Iran', 'Índia', 'Mexico',
                              'Argentina', 'Canada', 'Spain', 'Norway',
                              'Uruguay'])
not_manual = recent[~idx]

In [9]:
yesterday_wdt = yesterday.strftime("+%Y-%m-%dT00:00:00Z/11")
today_wdt = today.strftime("+%Y-%m-%dT00:00:00Z/11")

with open(f'../data/{today_table}.qs', 'w') as file:
    for index, row in not_manual.iterrows():
        print(
            row['item'] + "|P1603|" + str(int(row['Confirmed'])) +
            "|P585|" + yesterday_wdt + "|S854|" + '"' +
            "https://github.com/datasets/covid-19" + '"' +
            "|S813|" + today_wdt + "\n" +

            row['item'] + "|P1120|" + str(int(row['Deaths'])) +
            "|P585|" + yesterday_wdt + "|S854|" + '"' +
            "https://github.com/datasets/covid-19" + '"' +
            "|S813|" + today_wdt + "\n" +

            row['item'] + "|P8010|" + str(int(row['Recovered'])) +
            "|P585|" + yesterday_wdt + "|S854|" + '"' +
            "https://github.com/datasets/covid-19" + '"' +
            "|S813|" + today_wdt + "\n",
            file=file)

In [10]:
%run check_last_update_for_country_items.py

In [11]:
country_outbreak_items = list(recent["item"])

# Api only takes 50 at a time, so we have to cut it.


# implementation from    https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def get_chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))


chunks_of_country_outbreak_items = list(get_chunks(country_outbreak_items, 50))

outbreak_item_to_timestamp = {}

for chunk in chunks_of_country_outbreak_items:
    outbreak_item_to_timestamp.update(get_timestamp_of_last_edits(chunk))


6:80: E501 line too long (116 > 79 characters)


In [12]:
recent["timestamp_of_last_edit"] = recent["item"].map(outbreak_item_to_timestamp)

recent.head(1)

/home/lubianat/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid,timestamp_of_last_edit
144,2020-06-14,Afghanistan,24766,4725,471,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889,2020-06-15T01:22:58Z


1:80: E501 line too long (81 > 79 characters)


In [13]:
from datetime import datetime


def convert_timestamp_to_time_until_now(timestamp):

    time_in_datetime_format = datetime.strptime(timestamp,
                                                "%Y-%m-%dT%H:%M:%SZ")
    diff = datetime.now() - time_in_datetime_format
    return(diff)

In [14]:
recent["time_from_last_edit_until_now"] = recent["timestamp_of_last_edit"].map(convert_timestamp_to_time_until_now)

recent.head(1)

/home/lubianat/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid,timestamp_of_last_edit,time_from_last_edit_until_now
144,2020-06-14,Afghanistan,24766,4725,471,Q87768605,2020 coronavirus pandemic in Afghanistan,Q889,2020-06-15T01:22:58Z,08:08:46.997435


1:80: E501 line too long (115 > 79 characters)


In [15]:
outdated_items = recent[recent["time_from_last_edit_until_now"] > timedelta(hours=23)]

In [16]:
outdated_items.head(5)

,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid,timestamp_of_last_edit,time_from_last_edit_until_now
289,2020-06-14,Algeria,10919,7606,767,Q87202921,2020 coronavirus pandemic in Algeria,Q262,2020-06-08T17:00:32Z,6 days 16:31:12.997526
434,2020-06-14,Angola,140,61,6,Q88082534,2020 coronavirus pandemic in Angola,Q916,2020-05-29T04:41:32Z,17 days 04:50:12.997588
579,2020-06-14,Antigua and Barbuda,26,20,3,Q87708331,2020 coronavirus pandemic in Antigua and Barbuda,Q781,2020-05-14T21:22:34Z,31 days 12:09:10.997646
869,2020-06-14,Australia,7335,6851,102,Q83873548,2020 coronavirus pandemic in Australia,Q408,2020-05-28T22:20:24Z,17 days 11:11:20.997742
1014,2020-06-14,Austria,17109,16059,677,Q86847911,2020 coronavirus pandemic in Austria,Q40,2020-06-10T04:47:34Z,5 days 04:44:10.997769


In [17]:

table_date_in_wikidata_format = datetime.strptime(
    outdated_items["Date"].values[0], "%Y-%m-%d").strftime(
    "+%Y-%m-%dT00:00:00Z/11")

point_in_time = "|P585|" + table_date_in_wikidata_format

today_wdt = today.strftime("+%Y-%m-%dT00:00:00Z/11")


reference_URL = "|S854|" + '"' + "https://datahub.io/core/covid-19" + '"'
retrieved_in = "|S813|" + today_wdt
filename_in_archive = "|S7793|" + '"' + "r/countries-aggregated.csv" + '"'

reference = reference_URL + retrieved_in + filename_in_archive


with open(f'../data/{today_table}_outdated_items.qs', 'w') as file:
    for index, row in outdated_items.iterrows():
        print(
            row['item'] + "|P1603|" + str(int(row['Confirmed'])) +
            point_in_time + reference + "\n" +
            row['item'] + "|P1120|" + str(int(row['Deaths'])) +
            point_in_time + reference + "\n" +
            row['item'] + "|P8010|" + str(int(row['Recovered'])) +
            point_in_time + reference + "\n",
            file=file)

In [18]:
outdated_items.head(5)

,Date,Country,Confirmed,Recovered,Deaths,item,itemLabel,countryid,timestamp_of_last_edit,time_from_last_edit_until_now
289,2020-06-14,Algeria,10919,7606,767,Q87202921,2020 coronavirus pandemic in Algeria,Q262,2020-06-08T17:00:32Z,6 days 16:31:12.997526
434,2020-06-14,Angola,140,61,6,Q88082534,2020 coronavirus pandemic in Angola,Q916,2020-05-29T04:41:32Z,17 days 04:50:12.997588
579,2020-06-14,Antigua and Barbuda,26,20,3,Q87708331,2020 coronavirus pandemic in Antigua and Barbuda,Q781,2020-05-14T21:22:34Z,31 days 12:09:10.997646
869,2020-06-14,Australia,7335,6851,102,Q83873548,2020 coronavirus pandemic in Australia,Q408,2020-05-28T22:20:24Z,17 days 11:11:20.997742
1014,2020-06-14,Austria,17109,16059,677,Q86847911,2020 coronavirus pandemic in Austria,Q40,2020-06-10T04:47:34Z,5 days 04:44:10.997769


In [28]:
from wikidataintegrator import wdi_core, wdi_login
import credentials as credentials        
# login object
login_instance = wdi_login.WDLogin(user=credentials.username, pwd=credentials.password)


# data type object, e.g. for a NCBI gene entrez ID
death_counts_for_country = wdi_core.WDQuantity(value=451, prop_nr='P1120')
confirmed_counts_for_country = wdi_core.WDQuantity(value=451, prop_nr='P1603')
recovered_counts_for_country = wdi_core.WDQuantity(value=451, prop_nr='P8010')


# data goes into a list, because many data objects can be provided to 
data_to_update_for_country = [death_counts_for_country]
    
    # Search for and then edit/create new item
wd_item = wdi_core.WDItemEngine(wd_item_id="Q87768605",
                                data=data_to_update_for_country)
   
    
wd_item.write(login_instance,
              bot_account=True,
              edit_summary="updating case counts for today")

https://www.wikidata.org/w/api.php
Successfully logged in as CovidDatahubBot
2020-06-15 12:46:43.144571: maxlag. sleeping for 13.983333333333333 seconds


KeyboardInterrupt: 

2:34: W291 trailing whitespace
5:80: E501 line too long (87 > 79 characters)
14:70: W291 trailing whitespace
16:1: W293 blank line contains whitespace
17:5: E116 unexpected indentation (comment)
20:1: W293 blank line contains whitespace
21:1: W293 blank line contains whitespace


In [40]:
from datetime import datetime, date


references_for_statement = [[url_for_reference, reference_retrieved_in]]
url_for_reference = wdi_core.WDString(value="https://datahub.io/core/covid-19", prop_nr="P854", is_reference=True)


    
date_string = row["Date"]
date = datetime.strptime(date_string, '%Y-%M-%d')
date_string_in_wikidata_format = date.strftime("+%Y-%m-%dT00:00:00Z")


print(date_string_in_wikidata_format)
reference_retrieved_in = wdi_core.WDTime(date_string_in_wikidata_format, prop_nr='P813', is_reference=True)


references_list = [[url_for_reference, reference_retrieved_in]]

for index, row in outdated_items.iterrows():
    print(row)
    
    deaths = row["Deaths"]
    confirmeds = row["Confirmed"]
    recovereds = row["Recovered"]
    
    print(deaths)
    print(confirmeds)
    print(recovereds)
    
    deaths_statement = wdi_core.WDQuantity(value=deaths, prop_nr='P1120', references= references_list)
    confirmeds_statement = wdi_core.WDQuantity(value=confirmeds, prop_nr='P1603', references= references_list)
    recovereds_statement = wdi_core.WDQuantity(value=recovereds, prop_nr='P8010', references= references_list)
    
    data_to_update_for_country = [deaths_statement, confirmeds_statement, recovereds_statement]

    
    wd_item = wdi_core.WDItemEngine(wd_item_id="Q87768605",
                                    data=data_to_update_for_country)
   
    
    wd_item.write(login_instance,
                  bot_account=False,
                  edit_summary="updating case counts for today")
    
    
    
    break
    

+2020-01-14T00:00:00Z
Date                                                       2020-06-14
Country                                                       Algeria
Confirmed                                                       10919
Recovered                                                        7606
Deaths                                                            767
item                                                        Q87202921
itemLabel                        2020 coronavirus pandemic in Algeria
countryid                                                        Q262
timestamp_of_last_edit                           2020-06-08T17:00:32Z
time_from_last_edit_until_now                  6 days 16:31:12.997526
Name: 289, dtype: object
767
10919
7606
2020-06-15 13:12:52.419603: maxlag. sleeping for 11.15 seconds
2020-06-15 13:13:04.453025: maxlag. sleeping for 11.15 seconds
2020-06-15 13:13:16.344188: maxlag. sleeping for 12.066666666666666 seconds
2020-06-15 13:13:29.143074: maxlag. 

5:80: E501 line too long (114 > 79 characters)
8:1: W293 blank line contains whitespace
9:1: E303 too many blank lines (3)
10:1: F811 redefinition of unused 'date' from line 1
15:80: E501 line too long (107 > 79 characters)
22:1: W293 blank line contains whitespace
26:1: W293 blank line contains whitespace
30:1: W293 blank line contains whitespace
31:80: E501 line too long (102 > 79 characters)
31:86: E251 unexpected spaces around keyword / parameter equals
32:80: E501 line too long (110 > 79 characters)
32:94: E251 unexpected spaces around keyword / parameter equals
33:80: E501 line too long (110 > 79 characters)
33:94: E251 unexpected spaces around keyword / parameter equals
34:1: W293 blank line contains whitespace
35:80: E501 line too long (95 > 79 characters)
37:1: W293 blank line contains whitespace
38:5: E303 too many blank lines (2)
40:1: W293 blank line contains whitespace
41:1: W293 blank line contains whitespace
42:5: E303 too many blank lines (2)
45:1: W293 blank line conta

In [26]:
credentials

<module 'credentials' from '/home/lubianat/Documents/wikidata_covid19/sandbox/worldwide_data/src/credentials.py'>